In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("desafio modulo2") \
    .getOrCreate()

spark.version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
estabelecimentos = spark.read.csv('/content/drive/MyDrive/MBA/dados/estabelecimentos/estabelecimentos/', header=True, inferSchema=True, sep = ';')

estabelecimentos.printSchema()

In [ ]:
cnaes = spark.read.csv('/content/drive/MyDrive/MBA/dados/cnaes/cnaes.csv', header=True, inferSchema=True, sep = ';')

cnaes.printSchema()

In [ ]:
estabelecimentos.write.parquet("/content/drive/MyDrive/MBA/dados/estabelecimentos/parquet/estabelecimentos.parquet") 

In [ ]:
estabelecimentos.createOrReplaceTempView("estabelecimentos")

spark.sql("SELECT COUNT(*) FROM estabelecimentos").show()

In [ ]:
estabelecimentos.createOrReplaceTempView("estabelecimentos")

spark.sql("SELECT COUNT(*) FROM estabelecimentos WHERE LOGRADOURO IS NULL").show()

In [ ]:
def is_avenida(logradouro):
  if logradouro is not None: 
    logradouro = logradouro.upper()
    x = logradouro.startswith("AVENIDA")
    return x

In [ ]:
from pyspark.sql.functions import udf
spark.udf.register("is_avenida", is_avenida)

In [ ]:
estabelecimentos.createOrReplaceTempView("estabelecimentos")

spark.sql("SELECT COUNT(*) FROM estabelecimentos WHERE is_avenida(LOGRADOURO) == True").show()

In [ ]:
estabelecimentos.createOrReplaceTempView("estabelecimentos")

spark.sql("SELECT COUNT(DISTINCT(CEP)) FROM estabelecimentos WHERE CEP IS NOT NULL").show()

In [ ]:
cnaes.createOrReplaceTempView("cnaes")

spark.sql("SELECT COUNT(*) FROM cnaes").show()

In [ ]:
def is_cnae_cultivo(descricao_cnae):
  if descricao_cnae is not None:
    descricao_cnae = descricao_cnae.upper()
    x = descricao_cnae.startswith("CULTIVO")
    return x
  else:
    return False

In [ ]:
spark.udf.register("is_cnae_cultivo", is_cnae_cultivo)

In [ ]:
estabelecimentos_with_cnae = estabelecimentos.join(cnaes, estabelecimentos.CNAE_PRINCIPAL ==  cnaes.CNAE,"inner")

In [ ]:
estabelecimentos_with_cnae.printSchema()

In [ ]:
estabelecimentos_with_cnae.createOrReplaceTempView("estabelecimentos_with_cnae")

spark.sql("SELECT COUNT(*) from estabelecimentos_with_cnae WHERE is_cnae_cultivo(DESCRICAO_CNAE) == True").show()

In [ ]:
estabelecimentos.createOrReplaceTempView("estabelecimentos")
spark.sql("SELECT COUNT(*) from estabelecimentos WHERE IDENTIFICADOR_MATRIZ_FILIAL = 2").show()

In [ ]:
from pyspark.sql.functions import col

fire_df.groupBy('CallType').count().sort(col("count").desc()).show()

In [ ]:
spark.sql("SELECT CallType, COUNT(*) AS COUNT from fire_table  \
            GROUP BY CallType ORDER BY COUNT DESC").show()

In [ ]:
fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayinMins")
fire_df.createOrReplaceTempView("fire_table")


spark.sql("SELECT CallNumber, ResponseDelayInMins FROM fire_table \
            WHERE ResponseDelayInMins > 50").show()

